In [7]:
#
# Script to create a CSV file for the cats vs dogs dataset
#
import csv
import os

# Directory paths
cats_dir = "../data/train/cnn_train_images/cats"
dogs_dir = "../data/train/cnn_train_images/dogs"
csv_dir = '../data/train'

# File path within the folder
csv_file_path = os.path.join(csv_dir, "cats_vs_dogs_train.csv")

# List to store image paths and labels
data = []

# Process cats images
for filename in os.listdir(cats_dir):
    filepath = os.path.join(cats_dir, filename)
    
    # Splitting the string at the first occurrence of '.'
    id_number = filename.split('.', 1)

    # Keeping only the part before the first '.'
    id = id_number[0]

    data.append([id, 1, 0])  # Label 1 for cat

# Process dogs images
for filename in os.listdir(dogs_dir):
    filepath = os.path.join(dogs_dir, filename)
            
    # Splitting the string at the first occurrence of '.'
    id_number = filename.split('.', 1)

    # Keeping only the part before the first '.'
    id = id_number[0]

    data.append([id, 0, 1])  # Label 0 for non-hotdog

# Shuffle the data
import random
random.shuffle(data)

# Write data to CSV file
with open(csv_file_path, "w", newline="") as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(["Id", "Cat", "Dog"])
    csvwriter.writerows(data)


In [8]:
#
# Append labels from one CSV file to another (combine other train labels with cats vs dogs labels)
#

# Function to read CSV file and return a dictionary where keys are IDs
def read_csv_to_dict(file_path):
    data_dict = {}
    with open(file_path, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_dict[row['Id']] = row
    return data_dict

# Paths to the CSV files
csv1_path = "../data/train/train.csv"
csv2_path = "../data/train/cats_vs_dogs_train.csv"

# Read CSV files into dictionaries
csv1_data = read_csv_to_dict(csv1_path)
csv2_data = read_csv_to_dict(csv2_path)

# Append labels from CSV1 to CSV2 if IDs match
for id, row in csv2_data.items():
    if id in csv1_data:
        csv2_data[id].update(csv1_data[id])

# Write combined data to a new CSV file
combined_csv_path = "../data/train/cats_vs_dogs_train.csv"
with open(combined_csv_path, "w", newline='') as csvfile:
    fieldnames = list(csv2_data.values())[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(csv2_data.values())
